In [1]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import logging
import numpy as np
np.random.seed(42)

In [2]:
origin_column = 'CentroOrigen'
time_column = 'FechaLlegada'
destination_column = 'CentroDestino'
weight_column = 'Kilos'
volume_column = 'Palets'

zip_origin = 'PostalOrigen'
zip_destination = 'PostalEntrega'

In [3]:
raw = pd.read_parquet(r'DatosTransporte_ETL.parquet')
latitudes = pd.read_csv(r'listado-codigos-postales-con-LatyLon.csv', sep = ';', dtype={'codigopostalid': 'string'})
latitudes.rename(columns={'lat': 'lat1', 'lon': 'lat'}, inplace=True)
latitudes.rename(columns={'lat1': 'lon'}, inplace=True)

In [4]:
column_dict = {'origin_column': origin_column,
            'destination_column': destination_column,
            'time_column': time_column,
            'volume_column': volume_column,
            'weight_column': weight_column
            }


aggregation_type_dict = {volume_column: 'sum',
                        weight_column: 'sum'}

In [5]:

def get_kpi_summary(raw, time_column, origin_column, destination_column, frequency, aggregation_type_dict):
    ### cambiar columnas por diccionario
    raw[time_column] = pd.to_datetime(raw[time_column])
    raw.set_index(time_column, inplace=True)
    summary = raw.groupby([origin_column, destination_column]).resample(frequency).agg(operation_count = (origin_column, 'count'),
                                                                    volume = (volume_column, aggregation_type_dict[volume_column]),
                                                                    weight = (weight_column, aggregation_type_dict[weight_column])).reset_index()
    aggregated_columns = [k+'_'+aggregation_type_dict[k] for k in aggregation_type_dict]
    summary.rename(columns = dict(zip(['volume', 'weight'], aggregated_columns)), inplace=True)
    summary= summary[summary['FechaLlegada']=='2021-01-31']
    return summary

In [6]:
def get_node_coordinates(test, origin_column, zip_origin, destination_column, zip_destination):

    #### suponiendo que los CodOriden y fin son unicos
    #nodes = set(list(test[origin_column].unique()) + list(test[destination_column].unique()))

    origin_nodes = test[[origin_column, zip_origin]].drop_duplicates()
    origin_nodes = origin_nodes.merge(latitudes[['codigopostalid', 'lat', 'lon']].drop_duplicates(subset=['codigopostalid']), left_on=zip_origin, 
                                    right_on='codigopostalid')
    destination_nodes = test[[destination_column, zip_destination]].drop_duplicates()
    destination_nodes = destination_nodes.merge(latitudes[['codigopostalid', 'lat', 'lon']].drop_duplicates(subset=['codigopostalid']), 
                                            left_on=zip_destination, right_on='codigopostalid')

    origin_nodes.rename(columns = {origin_column: 'Node'}, inplace=True)
    destination_nodes.rename(columns = {destination_column: 'Node'}, inplace=True)

    node_atts = pd.concat([origin_nodes[['Node', 'lat', 'lon']], destination_nodes[['Node', 'lat', 'lon']]])

    if node_atts.shape[0]!=node_atts['Node'].nunique():
        node_atts.drop_duplicates(subset=['Node'], inplace=True)

    try:
        node_dict = node_atts.set_index('Node').to_dict(orient='index')
        return node_dict, node_atts
    except Exception as e:
        logging.error(msg=f'Could not convert to dictionary. The error is {str(e)}')
        return None

In [7]:
def get_node_kpis(summary, origin_column, destination_column,node_weight = 'operation_count'):
    try:
        inward_operations = summary.groupby(destination_column)[node_weight].sum().reset_index().rename(columns={destination_column: 'Node', node_weight: node_weight+'_inward'})
        outward_operations = summary.groupby(origin_column)[node_weight].sum().reset_index().rename(columns={origin_column: 'Node', node_weight: node_weight+'_outward'})

        total_operations = inward_operations.merge(outward_operations, on ='Node', how='outer').fillna(0)
        total_operations['total_'+node_weight] = total_operations[node_weight+'_inward'] + total_operations[node_weight+'_outward']

        kpi_node_atts = total_operations.set_index('Node').to_dict(orient='index')
        return kpi_node_atts
    except Exception as e:
        logging.error(msg=f'Could not convert KPIs to dictionary. The error is {str(e)}')
        return None
        

In [8]:
node_atts, node_df = get_node_coordinates(raw, origin_column, zip_origin, destination_column, zip_destination)
summary = get_kpi_summary(raw, time_column, origin_column, destination_column, 'M', aggregation_type_dict)
node_kpis = get_node_kpis(summary, origin_column, destination_column)

In [9]:
def build_graph(summary, node_atts, node_kpis):
    G = nx.from_pandas_edgelist(
    summary,
    source=origin_column,
    target=destination_column,
    edge_attr=True,
    create_using=nx.Graph())

    nx.set_node_attributes(G, node_atts)

    node_text = [str(int(node_kpis[node]['total_operation_count']))+' operations' for node in G.nodes]
    node_weights =  [int(node_kpis[node]['total_operation_count']) for node in G.nodes]
    kilos = summary['Kilos_sum'].values
    edge_texts = [str(k)+' kilograms' for k in kilos]

    return G, node_text, node_weights, edge_texts

In [10]:
G, node_text, node_weights, edge_texts = build_graph(summary, node_atts, node_kpis)

In [17]:
STOP

NameError: name 'STOP' is not defined

In [20]:
def plot_map_graph(G, node_text, node_weights, edge_texts):
    edge_lons = []
    edge_lats = []

    midpoints_lat = []
    midpoints_lon = []
    for edge in G.edges():
        lon_start, lat_start = G.nodes[edge[0]]['lon'], G.nodes[edge[0]]['lat']
        lon_end, lat_end = G.nodes[edge[1]]['lon'], G.nodes[edge[1]]['lat']
        edge_lons.extend([lon_start, lon_end, None])  
        edge_lats.extend([lat_start, lat_end, None])

        midpoint_lat = (lat_start + lat_end) / 2
        midpoint_lon = (lon_start + lon_end) / 2
        midpoints_lat.append(midpoint_lat)
        midpoints_lon.append(midpoint_lon)


    fig = go.Figure(go.Scattermapbox(
        mode="lines",
        lon=edge_lons,
        lat=edge_lats,
        hoverinfo='text',
        line=dict(width=1, color='#888'),
        showlegend=False
    ))

    
    fig.add_trace(go.Scattermapbox(
    mode='markers',
    lon=midpoints_lon,
    lat=midpoints_lat,
    marker=dict(
        size=5,  
        color='rgba(0,0,0,0)',  
        opacity=0  
    ),
    hoverinfo='text',
    text=edge_texts, 
))

    ############################## NODES ##############################
    node_latitudes = [G.nodes[node]['lat'] for node in G.nodes()]
    node_longitudes = [G.nodes[node]['lon'] for node in G.nodes()]
    fig.add_trace(go.Scattermapbox(
            mode="markers",
            lon=node_longitudes,
            lat=node_latitudes,
            text=node_text,
            hoverinfo='text', 
            marker=dict(
                showscale=True,
                colorscale='bluered',
                color=node_weights,
                colorbar=
                dict(
                    title="Node weight",
                    nticks=24,
                    tickfont=dict(color="#d8d8d8"),
                    titlefont=dict(color="#d8d8d8"),
                    thicknessmode="pixels"
                ),
                size=20),
                showlegend=False,
            textposition="bottom right"
        ))
    

    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=5,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox_center = {"lat": 43.42461111 , "lon": -3.522916667}
    )

    return fig

In [21]:
plot_map_graph(G, node_text, node_weights, edge_texts)

In [26]:
def map_app(G, node_text, node_weights, edge_texts):
    fig = plot_map_graph(G, node_text, node_weights, edge_texts)
    return fig

In [25]:
import gradio as gr
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            min_price = gr.Number(value=250, label="Minimum Price")
            max_price = gr.Number(value=1000, label="Maximum Price")
        boroughs = gr.CheckboxGroup(choices=["Queens", "Brooklyn", "Manhattan", "Bronx", "Staten Island"], value=["Queens", "Brooklyn"], label="Select Boroughs:")
        btn = gr.Button(value="Update Filter")
        map = gr.Plot()
    demo.load(plot_map_graph, [G,node_text, node_weights, edge_texts], map)
    btn.click(plot_map_graph, [G,node_text, node_weights, edge_texts], map)

demo.launch()

AttributeError: 'Graph' object has no attribute '_id'